# LSTM实践

循环神经网络的变种LSTM模型也有很多种结构，从常规的lstm，到各类复杂结构，这里根据实际使用情况，以pytorch为主，也会有tf2的例子，逐步记录。

## vanilla lstm

常规的lstm模型。

### LSTM in Torch

在之前了解过基本神经网络构建的基础上，现在来讨论循环神经网络的使用。

pytorch中，LSTM类继承自RNNBase类，RNNBase类继承自Module类，Module类是pytorch中完成一定网络功能的基类，可以通过继承该类定义自己的神经网络。自己实现神经网络时，一般要重写其forward方法。

Module实现了__call__方法，这意味着其可被当做可调用方法使用。比如下面有用到lstm()。

In [ ]:
"""一个简单的pytorch的lstm模型的示例"""
# Author: Robert Guthrie

import torch
import torch.autograd as autograd  # torch中自动计算梯度模块
import torch.nn as nn             # 神经网络模块
import torch.nn.functional as F   # 神经网络模块中的常用功能
import torch.optim as optim       # 模型优化器模块

torch.manual_seed(1)

lstm = nn.LSTM(3, 3)
# 生成一个长度为5，每一个元素为1*3的序列作为输入，这里的数字3对应于上句中第一个3
inputs = [autograd.Variable(torch.randn((1, 3))) for _ in range(5)]

# 设置隐藏层维度，初始化隐藏层的数据。hidden变量是一个元组，其第一个元素是LSTM隐藏层输出，另一个元素维护隐藏层的状态。
# torch.rand(1,1,3)就是生成了一个维度为(1,1,3)的以一定高斯分布生成的张量
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn((1, 1, 3))))

for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(out)
    print(hidden)

In [ ]:
"""代码来自博客https://machinelearningmastery.com/how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of
-household-power-consumption/ """
import numpy as np
import pandas as pd


# fill missing values with a value at the same time one day ago
def fill_missing(values):
    one_day = 60 * 24
    for row in range(values.shape[0]):
        for col in range(values.shape[1]):
            if np.isnan(values[row, col]):
                values[row, col] = values[row - one_day, col]


# load all data
dataset = pd.read_csv('../data/household_power_consumption.txt', sep=';', header=0, low_memory=False,
                      infer_datetime_format=True, parse_dates={'datetime': [0, 1]}, index_col=['datetime'])

# mark all missing values
dataset.replace('?', np.nan, inplace=True)
# make dataset numeric
dataset = dataset.astype('float32')
# fill missing
fill_missing(dataset.values)

# add a column for for the remainder of sub metering
values = dataset.values
dataset['sub_metering_4'] = (values[:, 0] * 1000 / 60) - (values[:, 4] + values[:, 5] + values[:, 6])

# save updated dataset
dataset.to_csv('../data/household_power_consumption.csv')

# resample minute data to total for each day
# load the new file
dataset = pd.read_csv('../data/household_power_consumption.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'],
                      index_col=['datetime'])
# resample data to daily
daily_groups = dataset.resample('D')
daily_data = daily_groups.sum()
# summarize
print(daily_data.shape)
print(daily_data.head())
# save
daily_data.to_csv('../data/household_power_consumption_days.csv')

In [ ]:
# univariate multi-step lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM


# split a univariate dataset into train/test sets
def split_dataset(data):
    # split into standard weeks
    train, test = data[1:-328], data[-328:-6]
    # restructure into windows of weekly data
    train = array(split(train, len(train) / 7))
    test = array(split(test, len(test) / 7))
    return train, test


# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        # calculate mse
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        # calculate rmse
        rmse = sqrt(mse)
        # store
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col]) ** 2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores


# summarize scores
def summarize_scores(name, score, scores):
    s_scores = ', '.join(['%.1f' % s for s in scores])
    print('%s: [%.3f] %s' % (name, score, s_scores))


# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
    # flatten data
    data = train.reshape((train.shape[0] * train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end < len(data):
            x_input = data[in_start:in_end, 0]
            x_input = x_input.reshape((len(x_input), 1))
            X.append(x_input)
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return array(X), array(y)


# train the model
def build_model(train, n_input):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    # define parameters
    verbose, epochs, batch_size = 1, 70, 16
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model


# make a forecast
def forecast(model, history, n_input):
    # flatten data
    data = array(history)
    data = data.reshape((data.shape[0] * data.shape[1], data.shape[2]))
    # retrieve last observations for input data
    input_x = data[-n_input:, 0]
    # reshape into [1, n_input, 1]
    input_x = input_x.reshape((1, len(input_x), 1))
    # forecast the next week
    yhat = model.predict(input_x, verbose=0)
    # we only want the vector forecast
    yhat = yhat[0]
    return yhat


# evaluate a single model
def evaluate_model(train, test, n_input):
    # fit model
    model = build_model(train, n_input)
    # history is a list of weekly data
    history = [x for x in train]
    # walk-forward validation over each week
    predictions = list()
    for i in range(len(test)):
        # predict the week
        yhat_sequence = forecast(model, history, n_input)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next week
        history.append(test[i, :])
    # evaluate predictions days for each week
    predictions = array(predictions)
    score, scores = evaluate_forecasts(test[:, :, 0], predictions)
    return score, scores


# load the new file
dataset = read_csv('../data/household_power_consumption_days.csv', header=0, infer_datetime_format=True,
                   parse_dates=['datetime'], index_col=['datetime'])
# split into train and test
train, test = split_dataset(dataset.values)
# evaluate model and get scores
n_input = 7
score, scores = evaluate_model(train, test, n_input)
# summarize scores
summarize_scores('lstm', score, scores)
# plot scores
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
pyplot.plot(days, scores, marker='o', label='lstm')
pyplot.show()

## stacked lstm

先对stacked lstm有个基本认识，参考：[Single Layer & Multi-layer Long Short-Term Memory (LSTM) Model with Intermediate Variables for Weather Forecasting](https://www.sciencedirect.com/science/article/pii/S187705091831439X)，一个典型的stacked lstm结构是这样的：

![](Picture9.png)

接下来参考[Stacked Long Short-Term Memory Networks](https://machinelearningmastery.com/stacked-long-short-term-memory-networks/)记录一些基本内容。

原始的lstm就是由一个LSTM隐含层和一个标准前向输出层组成。 Stacked LSTM 是对该模型的扩充，它有多个LSTM隐含层，每层有多个记忆单元。

### 为什么要增加深度

神经网络的深度通常被认为是该方法在一系列具有挑战性的预测问题上取得成功的原因。更多的隐含层能让神经网络更深。更深的隐含层目前被认为能从之前的层中重新组合学习到的特征并创建新的更高层次的抽象特征。比如从线到面到体。

一个足够深的神经网络结构能用来估计大部分函数。增加网络的深度提供了一种需要更少的神经元和更快的训练的解决方案。因此，增加深度是一种典型的优化方式。

### Stacked LSTM Architecture

因为LSTM是作用于序列数据，这意味着随着时间推移，层数的增加增加了输入观察的抽象级别。实际上就是将观察结果随时间分块或在不同的时间尺度上表示问题。

引用一段话：

" ... building a deep RNN by stacking multiple recurrent hidden states on top of each other. This approach potentially allows the hidden state at each level to operate at different timescale"

— [How to Construct Deep Recurrent Neural Networks, 2013](https://arxiv.org/abs/1312.6026)

在一些研究中，LSTM的深度比一个给定层的memory cells的个数对模型来说更重要。Stacked LSTMs 现在已经是序列预测问题的一种较稳定的技术。一个Stacked LSTM模型可以由一个LSTM模型和多个LSTM层构成。如下图所示，上面的LSTM层不输出一个值，而是输出一个序列到下一层LSTM。具体来说，每个输入时间步一个输出，而不是所有输入时间步一个输出。

### Implement Stacked LSTMs in Keras

在Keras中可以很容易的创建Stacked LSTM，每个LSTM memory cell 需要一个3D输入，当LSTM处理一个时间步长的输入序列时，每个存储单元将以2D数组的形式输出整个序列的单个值。

首先是一个单层的LSTM的模型：

In [1]:
# Example of one output for whole sequence
from keras.models import Sequential
from keras.layers import LSTM
from numpy import array
# define model where LSTM is also output layer
model = Sequential()
model.add(LSTM(1, input_shape=(3,1)))
model.compile(optimizer='adam', loss='mse')
# input time steps
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

ModuleNotFoundError: No module named 'keras'

可以看到输入数据有3个值。运行例子输出一个值，2D数组形式。

为了堆叠LSTM层，需要修改LSTM层输出一个3D的数组作为下一层的输入。设置return_sequences=True，它就会输出一个序列。

In [ ]:
# Example of one output for each input time step
from keras.models import Sequential
from keras.layers import LSTM
from numpy import array
# define model where LSTM is also output layer
model = Sequential()
model.add(LSTM(1, return_sequences=True, input_shape=(3,1)))
model.compile(optimizer='adam', loss='mse')
# input time steps
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

然后就可以增加LSTM层了。比如：

```python
model = Sequential()
model.add(LSTM(..., return_sequences=True, input_shape=(...)))
model.add(LSTM(...))
model.add(Dense(...))
```

或者更多层：

```python
model = Sequential()
model.add(LSTM(..., return_sequences=True, input_shape=(...)))
model.add(LSTM(..., return_sequences=True))
model.add(LSTM(..., return_sequences=True))
model.add(LSTM(...))
model.add(Dense(...))
```